# Lab 12: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

from controller import *

import nest_asyncio
nest_asyncio.apply()

2022-05-15 16:35:06,382 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

2022-05-15 16:35:09.199 Python[42949:1913886] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/org.python.python.savedState


# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [7]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        rc = RobotControl(ble)
        
        rc.spin_360(95)
        
        while rc.sensor_readings is None or len(rc.sensor_readings) <= 18:
            asyncio.run(asyncio.sleep(0.05))
        rc.stop_recording()
        
        # Collect & display the full sensor data
        cols = [
            "time", "dist_r", "dist_f", "acc_x", "acc_y", "acc_z", "gyr_x",
            "gyr_y", "gyr_z", "mag_x", "mag_y", "mag_z", "temp"
        ]
        sensor_data = pd.DataFrame(rc.sensor_arr, columns=cols)
        
        sensor_ranges = (np.flip(sensor_data["dist_f"][:18].to_numpy()))[np.newaxis].T
        sensor_ranges /= 1000
        print(sensor_ranges)
        
        return sensor_ranges, sensor_ranges


In [25]:
START_PLOTTER()

2022-05-15 16:39:53,154 | INFO     |: Creating New Plotter Process


2022-05-15 16:39:53.957 Python[43560:1919213] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/cb/bssy6wvs7yx5fqv9pd5x930w0000gn/T/org.python.python.savedState


In [24]:
STOP_PLOTTER()

2022-05-15 16:39:52,364 | INFO     |: Plotter is stopped


In [26]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2022-05-15 16:39:54,309 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:83:A5:6D:02:3C
2022-05-15 16:39:59,897 | INFO     |: Connected to C0:83:A5:6D:02:3C


In [27]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-15 16:40:01,276 | INFO     |:  | Number of observations per grid cell: 18
2022-05-15 16:40:01,277 | INFO     |:  | Precaching Views...


/Users/trueshot/Documents/git/fast-robots/lab12/ECE4960-sim-release/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2022-05-15 16:40:05,796 | INFO     |:  | Precaching Time: 4.519 secs
2022-05-15 16:40:05,797 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-15 16:40:05,798 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [28]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-15 16:40:07,061 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-15 16:40:07,062 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.443]
 [0.578]
 [0.462]
 [0.413]
 [0.442]
 [0.567]
 [0.825]
 [2.487]
 [2.792]
 [3.186]
 [0.576]
 [0.466]
 [2.847]
 [1.743]
 [0.649]
 [0.49 ]
 [0.414]
 [0.416]]
2022-05-15 16:40:17,924 | INFO     |: Update Step
2022-05-15 16:40:17,933 | INFO     |:      | Update Time: 0.003 secs
2022-05-15 16:40:17,935 | INFO     |: Bel index     : (10, 7, 10) with prob = 1.0
2022-05-15 16:40:17,936 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-15 16:40:17,938 | INFO     |: Belief        : (1.524, 0.914, 30.000)


In [29]:
loc.gaussian(loc.mapper.obs_views[:, :, :, 1] - loc.obs_range_data[1][0], 0, loc.sensor_sigma)

array([[[3.09315034e-004, 7.58498399e-004, 1.75954552e-004, ...,
         3.09315034e-004, 1.28834086e-004, 1.28834086e-004],
        [3.09315034e-004, 2.99472577e-003, 4.16174906e-001, ...,
         3.09315034e-004, 1.28834086e-004, 1.28834086e-004],
        [3.09315034e-004, 2.99472577e-003, 4.16174906e-001, ...,
         3.09315034e-004, 1.28834086e-004, 1.28834086e-004],
        ...,
        [5.56490962e-008, 5.56490962e-008, 5.56490962e-008, ...,
         5.56490962e-008, 5.56490962e-008, 5.56490962e-008],
        [5.56490962e-008, 5.56490962e-008, 5.56490962e-008, ...,
         5.56490962e-008, 5.56490962e-008, 5.56490962e-008],
        [5.56490962e-008, 5.56490962e-008, 5.56490962e-008, ...,
         5.56490962e-008, 5.56490962e-008, 5.56490962e-008]],

       [[2.39470054e-002, 7.58498399e-004, 1.75954552e-004, ...,
         8.82183830e-001, 5.23656960e-001, 5.23656960e-001],
        [8.82183830e-001, 9.82423686e-001, 6.58212416e-001, ...,
         8.82183830e-001, 5.23656960e-

2022-05-15 16:41:09,155 | INFO     |: Disconnected from B45C71B2-3BCA-639D-7AF3-3C14C2003C3E


In [25]:
loc.mapper.obs_views[:, :, :, 1]

array([[[0.17597636, 0.19894407, 0.16218069, ..., 0.17597636,
         0.15475102, 0.15475102],
        [0.17597636, 0.23709231, 0.44558779, ..., 0.17597636,
         0.15475102, 0.15475102],
        [0.17597636, 0.23709231, 0.44558779, ..., 0.17597636,
         0.15475102, 0.15475102],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.3048    , 0.19894407, 0.16218069, ..., 0.52792909,
         0.46425305, 0.46425305],
        [0.52792909, 0.59683221, 0.48654208, ..., 0.52792909,
         0.46425305, 0.46425305],
        [0.52792909, 0.71127693, 0.81090346, ..., 0.52792909,
         0.46425305, 0.46425305],
        ...,
        [0.        , 0.59683221, 0.48654208, ..., 0.        ,
         0.        , 0.        ],
        [0. 